# 改进的YOLO v1

backbone: ResNet_18

neck: SPP

head

In [3]:
# backbone: ResNet_18
import torch
import torch.nn as nn

class BasicBlock(nn.Module):

    def __init__(self, conv1_in, conv1_out, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(conv1_in, conv1_out, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(conv1_out)
        self.relu = nn.ReLU(inplace=True)
        
        self.conv2 = nn.Conv2d(conv1_out, conv1_out, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(conv1_out)
        
        # downsample是增加x通道数用的，因为下面一层的F(x)可能会增加通道数，前面一层无法直接相加
        self.downsample = downsample

    def forward(self, x):
        identity = x # 上一层的信息

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out) # conv-1有ReLU

        out = self.conv2(out)
        out = self.bn2(out) # 注意到这里conv-2没有ReLU

        if self.downsample is not None:
            identity = self.downsample(x) # 没记错的李沐课里说的是用1x1卷积来增加通道数

        out += identity
        out = self.relu(out) # 注意到，联合上一层信息之后在进行ReLU
        return out


class Backbone(nn.Module):
    def __init__(self):
        super(Backbone, self).__init__()

        # ResNet's Head
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNet Block-1
        self.layer1 = nn.Sequential(BasicBlock(64, 64), 
                                    BasicBlock(64, 64))

        # ResNet Block-2 
        downsample = nn.Sequential(nn.Conv2d(64, 128, kernel_size=1, stride=2, bias=False),
                                   nn.BatchNorm2d(128))

        self.layer2 = nn.Sequential(BasicBlock(64, 128, 2, downsample=downsample), 
                                    BasicBlock(128, 128))

        # ResNet Block-3
        downsample = nn.Sequential(nn.Conv2d(128, 256, kernel_size=1, stride=2, bias=False),
                                   nn.BatchNorm2d(256))
        self.layer3 = nn.Sequential(BasicBlock(128, 256, 2, downsample=downsample), 
                                    BasicBlock(256, 256))

        # ResNet Block-4
        downsample = nn.Sequential(nn.Conv2d(256, 512, kernel_size=1, stride=2, bias=False),
                                   nn.BatchNorm2d(512))
        self.layer4 = nn.Sequential(BasicBlock(256, 512, 2, downsample=downsample), 
                                    BasicBlock(512, 512))
        

    def forward(self, x):
        C_1 = self.conv1(x)
        C_1 = self.bn1(C_1)
        C_1 = self.relu(C_1)
        C_1 = self.maxpool(C_1)

        C_2 = self.layer1(C_1)
        C_3 = self.layer2(C_2)
        C_4 = self.layer3(C_3)
        C_5 = self.layer4(C_4)

        return C_5



# model = Backbone()
# #model.load_state_dict(torch.load('resnet18.pth'), strict=False)
# # print(model) # 打印模型的网络结构
# pre = torch.load('resnet18.pth') # 下载地址：https://download.pytorch.org/models/resnet18-5c106cde.pth
# pre = [k for k, v in pre.items()]
# # print(pre)
# model.state_dict().keys()
# model.load_state_dict(torch.load('resnet18.pth'), strict=False)

In [4]:
# neck: SPP
import torch
import torch.nn as nn

class Neck(nn.Module):
    def __init__(self):
        super(Neck, self).__init__()

    def forward(self, x):
        # backbone ouput size: 13*13*512
        # output size: (13-5+2*2)/1 + 1 = 13, channel=512
        x_1 = torch.nn.functional.max_pool2d(x, 5, stride=1, padding=2)
        # output size: (13-9+2*4)/1 + 1 = 13, channel=512
        x_2 = torch.nn.functional.max_pool2d(x, 9, stride=1, padding=4)
        # output size: (13-13+2*6)/1 + 1 = 13, channel=512
        x_3 = torch.nn.functional.max_pool2d(x, 13, stride=1, padding=6)
        # output size: 13*13, channel=512+512+512+512=2048
        x = torch.cat([x, x_1, x_2, x_3], dim=1) 

        # SPP -> Detection Head还需要用1*1卷积降维到512个Channel
        spp_to_head = nn.Sequential(nn.Conv2d(2048, 512, 1),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(0.1, inplace=True))

        return spp_to_head(x) 

In [7]:
# Detection Head
class Head(nn.Module):
    def __init__(self):
        super(Head, self).__init__()
        # SPP output size: 13*13*512
        # size: 13*13, channel: 256
        head1 = nn.Sequential(nn.Conv2d(512, 256, 1),
                                    nn.BatchNorm2d(256),
                                    nn.LeakyReLU(0.1, inplace=True))

        # size: (13-3+2*1)/1 + 1 = 13, channel: 512
        head2 = nn.Sequential(nn.Conv2d(256, 512, 3),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(0.1, inplace=True))

        # size: 13*13, channel: 256
        head3 = nn.Sequential(nn.Conv2d(512, 256, 1),
                                    nn.BatchNorm2d(256),
                                    nn.LeakyReLU(0.1, inplace=True))


        # size: (13-3+2*1)/1 + 1 = 13, channel: 512
        head4 = nn.Sequential(nn.Conv2d(256, 512, 3),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(0.1, inplace=True))

        self.head = nn.Sequential(head1, head2, head3, head4)

    def forward(self, x):
        return self.head(x)

In [9]:
# YOLOv1 Model
class YOLOv1(nn.Module):
    def __init__(self):
        super(YOLOv1, self).__init__()

        self.backbone = Backbone()
        self.neck = Neck()
        self.head = Head()

    def forward(self, x):
        
        out = self.backbone(x)
        out = self.neck(out)
        out = self.head(out)

        return out

model = YOLOv1()